## Trabalho Prático 2.
Bianca Gusmão Cordebello, Felipe Augusto Vaz Ferreira, Júlia Almeida Costa Regueiro - Tema C

## 1. Carregamento do dataset
1. Utilização de `from sklearn.datasets import load_breast_cancer` para importar o dataset
2. Atribuição do dataset à uma variável `data = load_breast_cancer`
3. exibição do dados

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
data = load_breast_cancer()
data

## 2. Análise Exploratória Básica
### a. Identificação de variáveis dependentes e independentes e o tipo delas

**Variáveis independentes:** mean radius, mean texture... 

**Tipo dado:** float

**Tipo variável:** numérica
<br><br>

**Variáveis dependentes:** target

**Tipo dado:** int

**Tipo variável:** numérica
<br><br>

**_Obs:_** Não há nenhuma variável categórica

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Separar as variáveis independentes (features)
X = df.drop('target', axis=1)

# Separar a variável dependente (alvo)
y = df['target']

#independente
print("Variáveis independentes: ", X.columns.tolist())

#dependente
print("Variáveis dependentes: ", y.name)

In [ ]:
X.dtypes

In [ ]:
y.dtypes

In [ ]:
# Selecionar colunas numéricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
print("Variáveis Numéricas:")
print(numeric_cols)

# Selecionar colunas categóricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print("Variáveis Categóricas:")
print(categorical_cols)

## b. Análise de métricas estatísticas como média, mediana e desvio-padrão das variáveis.
Para calcular a média usamos: `df.mean()`<br>
Para calcular a mediana usamos: `df.median()`<br>
Para calcular o desvio padrão usamos: `df.std()`<br>

In [ ]:
df.mean()

In [ ]:
df.median()

In [ ]:
df.std()

## c. Identificação de Outliers usando IQR

### Cálculo dos Quartis e IQR

Primeiro, calculamos o primeiro quartil (Q1) e o terceiro quartil (Q3) para cada variável no DataFrame `df`. O intervalo interquartil (IQR) é então calculado como a diferença entre Q3 e Q1:

`Q1 = df.quantile(0.25)`<br>
``Q3 = df.quantile(0.75)``<br>
``IQR = Q3 - Q1``<br>


Em seguida, definimos os limites para identificar outliers

Os limites inferior e superior para identificar outliers são definidos como 1.5 vezes o IQR abaixo de Q1 e acima de Q3:

Logo após, identificamos os outliers. O valor é considerado um outlier se estiver abaixo do limite inferior ou acima do limite superior:
`outliers = (df < limite_inferior) | (df > limite_superior)`

Por fim, para imprimir a quantidade de outliers em cada variável, removemos a coluna 'target' (variável dependente) 


In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = (df < limite_inferior) | (df > limite_superior)

# Contar outliers por variável
outliers_count = outliers.drop('target', axis=1).sum()

print("Número de outliers por variável:")
print(outliers_count)

## 3. Pré-processamento
*Impressão do **dataset atual** para comparação*

Foi escolhido a **normalização** e a **remoção de outliers** 

- Para a **normalização**:
    1. Importamos a classe "MinMaxScaler" da biblioteca scikit-learn. (importações no primeiro bloco de código)<br>
        `from sklearn.preprocessing import MinMaxScaler`

    2. Dimensionamos os dados de modo que todas as características estejam no intervalo [0, 1]<br>
        `scaler = MinMaxScaler(feature_range=(0, 1))`

    3. Transformamos todos os dados do DataFrame df para um intervalo específico (0 a 1) e criamos um novo DataFrame (dff) com esses dados transformados.<br>
        `dff = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)`
    
    4. Imprimimos para a visualização
        `print(dff)`



In [ ]:
df

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
dff = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
print(dff)

## Remoção de outliers do dataset normalizado
1. Importamos a biblioteca scipy que possui a função "zscore"  

2. Calculamos o z-score:<br>
    `z_scores = np.abs(stats.zscore(dff))`

3. Definimos um limiar para considerar outliers:<br>
    `threshold = 3`

4. Remoção dos outliers e visulização dos dados com e sem: <br>
    `df_no_outliers = dff[(z_scores < threshold).all(axis=1)]`<br>
    `print("Dados Originais:\n", df)`<br>
    `print("\nDados normalizados sem Outliers (Z-score):\n", df_no_outliers)`

*Obs: não utilizamos o one-hot-encoding porque ele é normalmente utilizado em variáveis categóricas e o dataset não possui*


In [ ]:
from scipy import stats

z_scores = np.abs(stats.zscore(dff))

threshold = 3

df_no_outliers = dff[(z_scores < threshold).all(axis=1)]
print("Dados Originais:\n", df)
print("\nDados normalizados sem Outliers (Z-score):\n", df_no_outliers)


#### Dataset Normalizado e sem outliers

In [ ]:
dff2 = pd.DataFrame(df_no_outliers)
dff2

## 4. Amostragem

1. Importamos a função "train_test_split" da biblioteca sklearn

2. Criamos um DataFrame "X_no_outlier" removendo a coluna 'target' do DataFrame dff2.
    `X_no_outlier = dff2.drop('target', axis=1)`

3. Separamos a variável dependente (alvo)
    `y_no_outlier = dff2['target']`

4. Dividimos os Dados em Conjuntos de Treinamento e Teste:<br>
    `X_train, X_test, y_train, y_test = train_test_split(X_no_outlier, y_no_outlier, test_size=0.1, random_state=2024)`<br><br>
    **Conjunto de Treinamento:** X_train, y_train;<br>
    **Conjunto de Teste:** X_test, y_test;<br>
    `test_size=0.1`: Define que 10% dos dados serão utilizados como conjunto de teste, ou seja, os outros 90% serão utilizados como conjunto de treinamento (X_train, y_train).


In [ ]:
from sklearn.model_selection import train_test_split
X_no_outlier = dff2.drop('target', axis=1)

y_no_outlier = dff2['target']

X_train, X_test, y_train, y_test = train_test_split(X_no_outlier, y_no_outlier, test_size=0.1, random_state=2024)
print(f"O conjunto de treinamento contem {X_train.shape[0]} amostras com {X_train.shape[1]} dimensoes")
print(f"O conjunto de teste contem {X_test.shape[0]} amostras com {X_test.shape[1]} dimensoes")

## 5. Modelo de Classificação
##### a. Árvore de Decisão
1. Importação da Classe DecisionTreeClassifier

2. Criamos uma instância do DecisionTreeClassifier, com uma semente.
    `classificador = DecisionTreeClassifier(random_state=2024)`

3. Logo após treinamos o "classificador" utilizando os dados de treinamento. ``classificador.fit(X_train, y_train).``




In [ ]:
from sklearn.tree import DecisionTreeClassifier

classificador = DecisionTreeClassifier(random_state=2024)
classificador.fit(X_train, y_train)

#### Visualização da árvore

In [ ]:
from sklearn import tree

tree.plot_tree(classificador)

## 6. Matriz de confusão

a. Para os dados de testes

1. Importamos a função "confusion_matrix" da biblioteca sklearn.

2. Utilizamos o modelo treinado (classificador) para prever os rótulos das amostras no conjunto de teste (X_test). E foi atribuído para uma variável "y_pred".<br>
    `y_pred = classificador.predict(X_test)`

3. Utilizamos `matrix = confusion_matrix(y_test, y_pred)` para calcular a matriz de confusão comparando o valor real (y_test) com valor previstos (y_pred).

b. A análise nos permitiu observar que ele acertou 43 e errou 7, ou seja, acertou mais que errou.



In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = classificador.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
matrix

## 7. Métricas de classificação
a. 
1. Importação da função "classification_report" da biblioteca sklearn.

2. Geração do relatório de classificação utilizando `report = classification_report(y_test, y_pred, target_names=data.target_names, zero_division=0)`, que compara os valores reais (y_test) com os previstos (y_pred)

**Análise**: Boas precisões, como por exemplo: 0.93 e 0.87. O recall também tem valores bons. Com isso, podemos considerar o modelo obtido aceitável.

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, target_names=data.target_names, zero_division=0)
print(report)
